In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_poisson_deviance, mean_squared_error

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer

import xgboost as xgb
import math

import joblib
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/french-motor-claims-datasets-fremtpl2freq/freMTPL2freq.csv


<div style="
  border-radius: 20px;
  padding: 25px;
  background: radial-gradient(circle at top, #cf1323, #000043);
  border: 2px solid #00ffc8;
  text-align: center;
  box-shadow: 0 0 25px rgba(0, 255, 250, 0.25);
">
  <h1 style="
    font-size: 28px;
    font-family: 'Trebuchet MS', sans-serif;
    letter-spacing: 2px;
    color: #00ffc8;
    text-shadow: 0 0 12px rgba(0,255,200,0.5);
  ">
    <span style="color:#7aa6ec5;">Load</span> <span style="color:#7aa6ec5;">the</span> <span style="color:#7aa6ec5;">DataFrame</span>
  </h1>
</div>

In [3]:
df = pd.read_csv('/kaggle/input/french-motor-claims-datasets-fremtpl2freq/freMTPL2freq.csv')
df = df.drop(['IDpol'], axis=1)

In [4]:
print('Dataset shape:', df.shape)
print('\nFirst 5 rows:')
df.head()

Dataset shape: (678013, 11)

First 5 rows:


,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1,0.10,D,5,0,55,50,B12,Regular,1217,R82
1,1,0.77,D,5,0,55,50,B12,Regular,1217,R82
2,1,0.75,B,6,2,52,50,B12,Diesel,54,R22
3,1,0.09,B,7,0,46,50,B12,Diesel,76,R72
4,1,0.84,B,7,0,46,50,B12,Diesel,76,R72


<div style="
  border-radius: 20px;
  padding: 25px;
  background: radial-gradient(circle at top, #cf1223, #000043);
  border: 2px solid #00ffc8;
  text-align: center;
  box-shadow: 0 0 25px rgba(0, 255, 200, 0.25);
">
  <h1 style="
    font-size: 28x;
    font-family: 'Trebuchet MS', sans-serif;
    letter-spacing: 2px;
    color: #00ffc8;
    text-shadow: 0 0 12px rgba(0,255,200,0.5);
  ">
    <span style="color:#7aa6ec5;">Exploratory</span> <span style="color:#7aa6ec2;">Data</span> <span style="color:#7aa6ec1;">Analysis</span>
  </h1>
</div>

The definitions of each column:

- ClaimNb: Claim numbers during exposure period
- Exposure: Period of exposure for a policy in year(s)
- VehPower: Car power, in categorical format
- VehAge: Car age, in continuous value format
- DrivAge: Driver’s age in years, in continuous value format
- BonusMalus: Bonus / Malus rating, <100 means bonus, >100 means malus, in continous value format
- VehBrand: Car brand, in categorical format
- VechGas: Fuel type, in categorical format
- Area: Density rating of the area car driver lives in, in categorical format
- Density: Density rating, number of inhabitants per km^2, in continous value format
- Region: Policy region in France, in categorical format

In [ ]:
df.info()

In [ ]:
df['Area'].value_counts()

In [ ]:
df.columns

In [5]:
cat_cols = df.select_dtypes(include='object').columns.to_list()
target = 'ClaimNb'
num_cols = df.select_dtypes(exclude='object').columns.drop(target).to_list()
base = ['Exposure', 'Area', 'VehPower', 'VehAge', 'DrivAge',
       'BonusMalus', 'VehBrand', 'VehGas', 'Density', 'Region']

In [17]:
print(cat_cols)

['Area', 'VehBrand', 'VehGas', 'Region']


In [ ]:
df.shape

In [32]:
print("ClaimNb value counts:")
df['ClaimNb'].value_counts().sort_index()

ClaimNb value counts:


ClaimNb
0     643953
1      32178
2       1784
3         82
4          7
5          2
6          1
8          1
9          1
11         3
16         1
Name: count, dtype: int64

## Missing any values

In [ ]:
df.isnull().sum()

#### There is no missing values in the dataset

In [10]:
df.describe()

,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,Density
count,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000
mean,0.053247,0.528750,6.454631,7.044265,45.499122,59.761502,1792.422405
std,0.240117,0.364442,2.050906,5.666232,14.137444,15.636658,3958.646564
min,0.000000,0.002732,4.000000,0.000000,18.000000,50.000000,1.000000
25%,0.000000,0.180000,5.000000,2.000000,34.000000,50.000000,92.000000
50%,0.000000,0.490000,6.000000,6.000000,44.000000,50.000000,393.000000
75%,0.000000,0.990000,7.000000,11.000000,55.000000,64.000000,1658.000000
max,16.000000,2.010000,15.000000,100.000000,100.000000,230.000000,27000.000000


# Feature Engineering

### We will also simplify the data for our GLM model. In particular, we will adjust the following columns:

- ClaimNb: cap at 4 claims
- VehAge: cap at 20 years
- DrivAge: cap at 90 years old
- BonusMalus: cap at 150, round to nearest integer
- Density: apply log
- Exposure: cap at 1 year

and subsequently add the following 'GLM' columns that will be used for model fitting:
- AreaGLM: convert alphabet into integer
- VehPowerGLM: cap at 9
- VehAgeGLM: create 3 bins
- DrivAgeGLM: create 7 bins


One of the reasons for capping variables is that insurance claim counts are highly skewed: most policyholders have 0-2 claims, but a few may have 10+. Extreme values like 20 claims are rare, often due to data errors, fraud, etc. 

In [6]:
def fe(df):
    
    df_freq = df.copy()

    # Clipping/ capping 
    
    df_freq['ClaimNb'] = df_freq['ClaimNb'].clip(upper=4)
    df_freq['VehAge'] = df_freq['VehAge'].clip(upper=20)
    df_freq['DrivAge'] = df_freq['DrivAge'].clip(upper=90)
    df_freq['Exposure'] = df_freq['Exposure'].clip(upper=1.0)

    # Conditional capping
    df_freq['BonusMalus'] = np.where(df_freq['BonusMalus'] > 150, 150, df_freq['BonusMalus']).astype(int)
    df_freq['VehPowerGLM']= np.where(df_freq['VehPower'] > 9, 9, df_freq['VehPower']).astype(str)

    # Log transform
    df_freq['Log_Density'] = np.log1p(df_freq['Density'])
    
    df_freq['AreaGLM']=df_freq['Area'].apply(lambda x: ord(x)-64)
    
    df_freq['VehAgeGLM']=pd.cut(df_freq['VehAge'], bins=[0,1,10,np.inf], labels=[1,2,3], include_lowest=True)
    df_freq['DrivAgeGLM']=pd.cut(df_freq['DrivAge'], bins=[18,21,26,31,41,51,71,np.inf], labels=[1,2,3,4,5,6,7], include_lowest=True)
    df_freq['BonusMalusGLM'] = df_freq['BonusMalus']
    df_freq['DensityGLM'] = df_freq['Density']

    original_cols = df.columns.tolist()

    fe_cols = [c for c in df_freq.columns if c not in original_cols]

    return df_freq, fe_cols

df_freq, fe_cols = fe(df)
print(fe_cols)

['VehPowerGLM', 'Log_Density', 'AreaGLM', 'VehAgeGLM', 'DrivAgeGLM', 'BonusMalusGLM', 'DensityGLM']


In [ ]:
df_freq.columns

In [ ]:
df.columns

In [13]:
df_freq.dtypes

ClaimNb             int64
Exposure          float64
Area               object
VehPower            int64
VehAge              int64
DrivAge             int64
BonusMalus          int64
VehBrand           object
VehGas             object
Density             int64
Region             object
VehPowerGLM        object
Log_Density       float64
AreaGLM             int64
VehAgeGLM        category
DrivAgeGLM       category
BonusMalusGLM       int64
DensityGLM          int64
dtype: object

## Count Encoding

In [7]:
cat_col2 = ['VehBrand', 'VehGas', 'Region', 'VehPowerGLM', 'VehAgeGLM', 'DrivAgeGLM']
num_cols = ['BonusMalusGLM', 'Density', 'Log_Density', 'AreaGLM', 'DensityGLM']

In [8]:
ce_cols = []
for col in cat_col2:
    new_cols = f'{col}_CE'
    df_freq[new_cols] = df_freq[col].map(df_freq[col].value_counts())
    ce_cols.append(new_cols)

In [27]:
print(ce_cols)

['Area_CE', 'VehBrand_CE', 'VehGas_CE', 'Region_CE', 'VehPowerGLM_CE', 'VehAgeGLM_CE', 'DrivAgeGLM_CE']


In [28]:
df_freq.head()

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,...,DrivAgeGLM,BonusMalusGLM,DensityGLM,Area_CE,VehBrand_CE,VehGas_CE,Region_CE,VehPowerGLM_CE,VehAgeGLM_CE,DrivAgeGLM_CE
0,1,0.10,D,5,0,55,50,B12,Regular,1217,...,6,50,1217,151596,166024,345877,84752,124821,129023,186606
1,1,0.77,D,5,0,55,50,B12,Regular,1217,...,6,50,1217,151596,166024,345877,84752,124821,129023,186606
2,1,0.75,B,6,2,52,50,B12,Diesel,54,...,6,50,54,75459,166024,332136,7994,148976,363208,186606
3,1,0.09,B,7,0,46,50,B12,Diesel,76,...,5,50,76,75459,166024,332136,31329,145401,129023,165224
4,1,0.84,B,7,0,46,50,B12,Diesel,76,...,5,50,76,75459,166024,332136,31329,145401,129023,165224


In [ ]:
df_freq.describe()

In [ ]:
df_freq.columns

## Preparing data for Modeling

In [9]:
features = ce_cols + num_cols
print(len(features))

11


In [10]:
X = df_freq[features]
y = df_freq['ClaimNb']
exposure = df_freq['Exposure']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Exposure shape: {exposure.shape}")

Features shape: (678013, 11)
Target shape: (678013,)
Exposure shape: (678013,)


## Train-test-split

In [11]:
# Split the data (stratify not needed for regression)
X_train, X_test, y_train, y_test, exposure_train, exposure_test = train_test_split(
    X, y, exposure, test_size=0.2, random_state=42
)

## Poisson Regression Modeling

In [13]:
poisson_model = PoissonRegressor(
    alpha = 1e-6,
    solver='newton-cholesky',
    max_iter=1200
).fit(X_train, y_train, sample_weight=exposure_train)

In [14]:
y_pred_train = poisson_model.predict(X_train)
y_pred_test = poisson_model.predict(X_test)

# Since we used sample_weight, predictions are expected frequency per unit exposure
# Multiply by exposure to get expected claim count
expected_claims_train = y_pred_train * exposure_train
expected_claims_test = y_pred_test * exposure_test

print("Poisson Regression Results:")
print(f"Training Poisson deviance: {mean_poisson_deviance(y_train, y_pred_train, sample_weight=exposure_train):.4f}")
print(f"Test Poisson deviance: {mean_poisson_deviance(y_test, y_pred_test, sample_weight=exposure_test):.4f}")
print(f"Training MSE: {mean_squared_error(y_train, expected_claims_train):.4f}")
print(f"Test MSE: {mean_squared_error(y_test, expected_claims_test):.4f}")

Poisson Regression Results:
Training Poisson deviance: 0.3574
Test Poisson deviance: 0.3591
Training MSE: 0.0563
Test MSE: 0.0564


In [18]:
# Get coefficients from the model
coefficients = poisson_model.coef_
intercept = poisson_model.intercept_

# Create a dataframe of feature importance
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': coefficients,
    'exp_Coefficient': np.exp(coefficients)
}).sort_values('Coefficient', ascending=False)

print("Top 10 positive coefficients (increase claim rate):")
print(feature_importance.head(10).to_string())

print(f"\nBaseline rate (intercept exp): {np.exp(intercept):.4f} claims/year")

Top 10 positive coefficients (increase claim rate):
           Feature   Coefficient  exp_Coefficient
8      Log_Density  6.692872e-02         1.069219
6    BonusMalusGLM  1.819133e-02         1.018358
2        Region_CE  1.371277e-06         1.000001
3   VehPowerGLM_CE  1.106800e-06         1.000001
5    DrivAgeGLM_CE  1.010086e-06         1.000001
4     VehAgeGLM_CE  4.100396e-07         1.000000
0      VehBrand_CE -3.212100e-08         1.000000
1        VehGas_CE -1.768225e-06         0.999998
7          Density -2.330137e-06         0.999998
10      DensityGLM -2.330699e-06         0.999998

Baseline rate (intercept exp): 0.0176 claims/year


### Insights from coefficient of each feature

- Log density has the strongest coefficient since the denser area raise claims 6.9% per unit. For example, city driver in Delhi (high density) might have 7% higher expected claims than rural—more traffic, more mishaps.
- BonusMalusGLM (higher score means with more past claims/ penalties) has a mild influence on the claim rate. Example: A driver with penalties (high score) is ~2% riskier—insurers hike premiums for repeat offenders.
- Rest of the features have tiny effect on the claim rate.

## Saving the model for download

In [22]:
with open('frequency_model.pkl', 'wb') as f:
    pickle.dump(poisson_model, f)

# Save feature information
feature_info = {
    'categorical_features': ce_cols,
    'numerical_features': num_cols,
    'feature_names': X_train.columns,
    'baseline_rate': float(np.exp(poisson_model.intercept_)) if 'poisson_model' in dir() else 0.1
}

with open('feature_info.pkl', 'wb') as f:
    pickle.dump(feature_info, f)

print("Models saved! Download these files for local development.")

Models saved! Download these files for local development.
